In [1]:
from sys import byteorder

byteorder = 'big'


以下サイトを参考にしました。  
https://note.com/cider00/n/nf2b49924b13e

In [2]:
def extract_between_strings(filename, string_a, string_b):
    """
    extract strings between string_a and string_b from file
    """
    results = []
    capture = False
    buffer = []
    
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            if string_a in line:
                capture = True
                buffer.append(line)
                continue
            
            if capture:
                buffer.append(line)
                if string_b in line:
                    capture = False
                    results.append(''.join(buffer))
                    buffer = []

    return results

In [3]:
def to_hex(vcs_string, suffix_to_remove):
    # assume string as something like =89=C6=8C=76=95=EB\nLOCATION
    a = vcs_string.replace("=","")
    b = a.replace("\n","")
    c = b.replace(suffix_to_remove,"")
    return c

def decode_string(preprocessed_string, encoding="shift-jis"):
    a = int(preprocessed_string, 16)
    b = a.to_bytes(len(str(a)), 'big')
    d = ""
    try:
        c = b.decode(encoding=encoding)
        d = c.strip("\x00")
    except UnicodeDecodeError as e:
        try:
            c = b.decode(encoding="cp932")
            d = c.strip("\x00")
        except UnicodeDecodeError as e2:
            print("catch UnicodeDecodeError. can't decode with shift-jis and cp932")
    finally:
        return d
    
        

#### DESCRITIONとCATEGORIESに囲まれた部分をまず取り出す

In [4]:
filename = 'data/[your_file].vcs'
string_a = 'DESCRIPTION'
string_b = 'CATEGORIES'
extracted_strings_body = extract_between_strings(filename, string_a, string_b)

# 出力（必要に応じて）
for s in extracted_strings_body:
    print(s)

DESCRIPTION;CHARSET=SHIFT_JIS;ENCODING=QUOTED-PRINTABLE:=30=38=0A=0A=0A=0A=0A=0A=83=58=83=7D=83=7A=31=32=39=38=89=7E=2F=8C=8E=
=20=93=F1=94=4E=8A=D4=0A=0A=83=43=83=49=83=93=83=82=83=6F=83=43=83=8B=
=20=38=35=30=89=7E=2F=8C=8E=0A=63=68=61=74=67=70=74=20=70=6C=75=73=20=
=32=30=83=68=83=8B=2F=8C=8E=0A=0A=73=70=65=61=6B=20=32=30=32=33=30=36=
=30=35=20=94=4E=8D=58=90=56=31=32=38=30=30=0A=0A=6E=65=74=66=6C=69=78=
=0A=32=30=32=33=30=34=31=39=89=F0=96=F1=0A=0A=95=73=93=AE=8E=59=8E=E6=
=93=BE=90=C5=20=36=8C=8E=82=B2=82=EB=0A=0A=0A=0A=92=99=8B=E0=88=EA=94=
=CA=82=CC=8C=B3=81=58=82=CC=83=78=81=5B=83=58=0A=34=38=30=30=30=30=0A=
=0A=61=6D=61=7A=6F=6E=20=6B=69=64=73=32=30=32=34=30=33=8D=58=90=56=20=
=8C=8E=34=38=30=89=7E=0A=0A=0A=0A=88=DA=93=AE=8D=CF=0A=31=30=30=8A=79=
=93=56=8F=D8=8C=94=0A=31=30=63=6F=69=6E=63=68=65=63=6B=0A=0A=0A=8F=AC=
=8A=77=8D=5A=82=C6=90=B6=8B=A6=82=CD=97=58=92=99=88=F8=82=AB=97=8E=82=
=C6=82=B5=0A=8D=C8=8E=71=95=DB=8C=AF=82=CD=73=62=69=88=F8=82=AB=97=8E=
=82=C6=82=B5=0A=0A=0A

In [5]:
# 題名部分
string_c = 'SUMMARY'
string_d = 'LOCATION'
extracted_strings_title = extract_between_strings(filename, string_c, string_d)



### 1行ごとに抜き出していく

In [18]:
text_dic = {}

OUTPUT_FOLDER = "output"

import os
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for title, body in zip(extracted_strings_title, extracted_strings_body):
    # 中身は:で区切った二つ目
    title_str = title.split(":")[1]
    body_str = body.split(":")[1]
    # titleはLOCATION, bodyはCAGORIRIESを消す
    title_hex = to_hex(title_str, "LOCATION")
    if title_hex == "" : continue
    body_hex = to_hex(body_str, "CATEGORIES")
    # 中身がなくてもタイトルだけのtextファイルを保存する
    if body_hex == "" :
        title_text = decode_string(title_hex)
        text_dic[title_text] = ""
    else:
        body_text = decode_string(body_hex)
        title_text = decode_string(title_hex)
        text_dic[title_text] = body_text

        


print(text_dic)

    

catch UnicodeDecodeError. can't decode with shift-jis and cp932
catch UnicodeDecodeError. can't decode with shift-jis and cp932
catch UnicodeDecodeError. can't decode with shift-jis and cp932
{'家計簿': '08\n\n\n\n\n\nスマホ1298円/月 二年間\n\nイオンモバイル 850円/月\nchatgpt plus 20ドル/月\n\nspeak 20230605 年更新12800\n\nnetflix\n20230419解約\n\n不動産取得税 6月ごろ\n\n\n\n貯金一般の元々のベース\n480000\n\namazon kids202403更新 月480円\n\n\n\n移動済\n100楽天証券\n10coincheck\n\n\n小学校と生協は郵貯引き落とし\n妻子保険はsbi引き落とし\n\n\n保険\n自動車シエンタ4千円弱/月\n入院保険2428→2497円/月\n毎月27日引き落とし\nこれらはsbi引き落とし20211027より\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', '論文要約スクリプトの確認': '', '1028(土)学習発表会': '', '0909アップルフェスタ': '同じ日に授業参観', '0901 懇親会': 'chain\nビル一階月と太陽', '番号 銀行、お金': 'MCJ\nお客様ID ：220900361241\nパスワード：M3i2\n\n住 my note\nmoro.qq82@gmail.com\nh4i2\n\n住信SBIネット銀行\nブドウ102\n3365459\n262137\n+b5i2\nb34\nmoroqq82\n\n東京スター銀行\ntsbMYaQR\nmoro-pc-2\n3731\n第二 5852\ns3i\nおおひら\n在良\n熊谷\n\n三井住友カード\nハンドルネーム moroqq\nmoro-pc-2\nmoroqq82\na3\n4741\n\n三井住友\n6900040006332112  409 07

In [20]:
for title in text_dic:
    with open(OUTPUT_FOLDER + "/" + title + ".txt", mode="w", encoding="shift-jis") as f:
        f.write(text_dic[title])

### utf-8で保存しなおす

In [21]:
import glob

text_path_list = glob.glob("output/*.txt")

for text_path in text_path_list:
    with open(text_path, mode="r", encoding="shift-jis") as f:
        item = f.read()
    with open(text_path, mode="w", encoding="utf-8") as f:
        f.write(item)

